<a href="https://colab.research.google.com/github/Bhavya232003/Deep-Learning-Codes/blob/main/CNNipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten_size = 8 * 14 * 14  # Corrected for MNIST (28x28 -> 14x14 after pooling)
        self.fc = nn.Linear(self.flatten_size, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv(x)))
        x = x.view(x.size(0), -1)  # Flatten dynamically
        x = self.fc(x)
        return torch.log_softmax(x, dim=1)

# Load and preprocess data
def get_data_loader(train=True):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    dataset = datasets.MNIST(root='./data', train=train, download=True, transform=transform)
    data_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    return data_loader

# Training function
def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    train_loader = get_data_loader(train=True)

    for epoch in range(5):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

    return model

# Accuracy calculation function
def evaluate_model(model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    test_loader = get_data_loader(train=False)
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Train and evaluate the model
model = train_model()
evaluate_model(model)


Epoch 1, Loss: 0.1715
Epoch 2, Loss: 0.1544
Epoch 3, Loss: 0.0063
Epoch 4, Loss: 0.0157
Epoch 5, Loss: 0.0151
Accuracy: 97.94%
